In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
def update(company):
    
#     Request from API Alpha Vantage
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={company}&apikey=JN45H11CRRH8ZZW4'

    api_data = requests.get(url)
    api_data = api_data.json()

    api_data = api_data["Time Series (Daily)"]
    
#     Create DataFrame
    api_df = pd.DataFrame(api_data)
    api_df = api_df.transpose()
    api_df.index = pd.to_datetime(api_df.index)
    api_df.sort_index(inplace=True)
    api_df.index.name = 'timestamp'
    api_df.rename(columns={'1. open': 'open', '2. high': 'high', '3. low': 'low', '4. close': 'close', '5. volume': 'volume'}, inplace=True)
    
#     Fetch CSV
    csv_df = pd.read_csv(f'{company}.csv', parse_dates=['timestamp'])
    csv_df.set_index('timestamp', inplace=True)
    csv_df.sort_index(inplace=True)
    csv_df[['open', 'high', 'low', 'close', 'volume']] = csv_df[['open', 'high', 'low', 'close', 'volume']].astype('object')
    
#     Drop Overlapping Data
    for index in api_df.index:
        if index in csv_df.index:
            csv_df.drop(index, inplace=True)
    
#     Concat CSV and API Data
    new_df = pd.concat([csv_df, api_df])
    
# Update CSV
    new_df.to_csv(f'{company}.csv')

In [3]:
update('GOOG')

In [4]:
pd.read_csv('GOOG.csv').tail()

,timestamp,open,high,low,close,volume
1460,2019-10-21,1252.26,1254.629,1240.600,1246.15,1027200
1461,2019-10-22,1247.85,1250.600,1241.380,1242.80,1023800
1462,2019-10-23,1242.36,1259.890,1242.360,1259.13,911500
1463,2019-10-24,1260.90,1264.000,1253.715,1260.99,1028100
1464,2019-10-25,1251.03,1269.600,1250.010,1265.13,1210100
